In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import re
import random
from datetime import datetime
from datetime import timedelta
import time
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# reformat the CSV to use | instead of , to separate rows, then import [a selection] of the data as a dataframe
for_pd = StringIO()
with open('../data/accre-jobs-2020.csv') as accre:
    for line in accre:
        new_line = re.sub(r',', '|', line.rstrip(), count=12)
        print (new_line, file=for_pd)

for_pd.seek(0)

accre = pd.read_csv(for_pd, sep='|')[1000000:1005000]
print (accre.head())

            JOBID ACCOUNT      USER  REQMEM   USEDMEM     REQTIME  USEDTIME  NODES  CPUS   PARTITION EXITCODE      STATE NODELIST
1000000  18616746     cms  cmslocal  5000Mn  1284.14M  2-00:00:00  00:15:35      1     1  production      0:0  COMPLETED   cn1423
1000001  18616752     cms  cmslocal  5000Mn  1275.16M  2-00:00:00  00:11:41      1     1  production      0:0  COMPLETED   cn1424
1000002  18616754     cms  cmslocal  5000Mn  1259.60M  2-00:00:00  00:15:35      1     1  production      0:0  COMPLETED   cn1311
1000003  18616757     cms  cmslocal  5000Mn  1271.18M  2-00:00:00  00:15:36      1     1  production      0:0  COMPLETED   cn1311
1000004  18616758     cms  cmslocal  5000Mn  1260.83M  2-00:00:00  00:15:35      1     1  production      0:0  COMPLETED    cn372


In [4]:
# ALTERNATIVE METHOD:
# To fully randomize the data, we could do the following:
# import pandas as pd
# import random

# The data to load
# f = "my_data.csv"

# Count the lines
# num_lines = sum(1 for l in open(for_pd))

# Sample size - in this case ~10%
# size = int(num_lines / 10)

# The row indices to skip - make sure 0 is not included to keep the header!
# skip_idx = random.sample(range(1, num_lines), num_lines - size)

# Read the data
# data = pd.read_csv(f, skiprows=skip_idx, ... )

In [5]:
# Take a look at the columns in the dataframe
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 1000000 to 1004999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      5000 non-null   object
 1   ACCOUNT    5000 non-null   object
 2   USER       5000 non-null   object
 3   REQMEM     5000 non-null   object
 4   USEDMEM    5000 non-null   object
 5   REQTIME    5000 non-null   object
 6   USEDTIME   5000 non-null   object
 7   NODES      5000 non-null   int64 
 8   CPUS       5000 non-null   int64 
 9   PARTITION  5000 non-null   object
 10  EXITCODE   5000 non-null   object
 11  STATE      5000 non-null   object
 12  NODELIST   5000 non-null   object
dtypes: int64(2), object(11)
memory usage: 507.9+ KB


In [6]:
# Look at some of the values and how often they appear in the dataframe:
print(accre.PARTITION.value_counts())
print(accre.STATE.value_counts())

production    4402
nogpfs         492
sam             78
pascal          22
maxwell          6
Name: PARTITION, dtype: int64
COMPLETED    4997
CANCELLED       3
Name: STATE, dtype: int64


### Data Cleaning

First, clean up and standardize the ``REQMEM`` and ``USEDMEM`` columns so that they contain both values per CPU.

In [7]:
# Grab a test string to test the RegExes
test_split = accre.loc[1000000,'REQMEM']
print(test_split)
# Use the re.findall() method to get the number for REQMEM
print(re.findall(r'\d+', test_split))
# Get the unit of measure for REQMEM
print(re.findall(r'..$', test_split))
# alternatively:
print(re.findall(r'\D+', test_split))

5000Mn
['5000']
['Mn']
['Mn']


In [8]:
# Create a new column REQMEM_NO which only contains the number
accre['REQMEM_NO'] = accre['REQMEM'].str.extract('(\d+)')
# Create a new column REQMEM_TYPE which only contains the type
accre['REQMEM_TYPE'] = accre['REQMEM'].str.extract('(\D+)')
# Convert REQMEM_NO to a mathable data type
accre['REQMEM_NO'] = pd.to_numeric(accre['REQMEM_NO'])

In [9]:
# Slice the dataframe by REQMEM_TYPE to get some sample values to test
print(accre[accre['REQMEM_TYPE'] == 'Mc'].head(50))
print(accre[accre['REQMEM_TYPE'] == 'Mn'].head(50))

               JOBID   ACCOUNT     USER  REQMEM   USEDMEM     REQTIME  USEDTIME  NODES  CPUS   PARTITION EXITCODE      STATE NODELIST  REQMEM_NO REQMEM_TYPE
1000006     18616763  cmsadmin  autocms  2048Mc   828.82M    12:00:00  00:05:08      1     1  production      0:0  COMPLETED   cn1122       2048          Mc
1000054     18616942  cmsadmin  autocms  2048Mc   860.54M    12:00:00  00:05:25      1     1  production      0:0  COMPLETED    cn430       2048          Mc
1000165     18617189   pasilla  earlean  3072Mc   197.12M  2-00:00:00  00:03:07      1    10  production     0:15  CANCELLED   cn1459       3072          Mc
1000166     18617190  cmsadmin  autocms  2048Mc   817.76M    12:00:00  00:02:42      1     1  production      0:0  COMPLETED   cn1133       2048          Mc
1000203   18617244_1    casaba     anne  8192Mc  1010.52M    02:30:00  02:30:15      1     1  production      0:0  COMPLETED   cn1335       8192          Mc
1000204   18617244_2    casaba     anne  8192Mc  1022.70M 

In [10]:
# Create a test variable for each data type for the if-elif statement
test_mn = accre.loc[1000000, ['REQMEM_NO', 'REQMEM', 'NODES', 'CPUS', 'REQMEM_TYPE']]
test_mn['CPUS'] = 5
print(test_mn)
test_mc = accre.loc[1000006, ['REQMEM_NO', 'REQMEM', 'NODES', 'CPUS', 'REQMEM_TYPE']]
test_mc['CPUS'] = 5
print(test_mc)

REQMEM_NO        5000
REQMEM         5000Mn
NODES               1
CPUS                5
REQMEM_TYPE        Mn
Name: 1000000, dtype: object
REQMEM_NO        2048
REQMEM         2048Mc
NODES               1
CPUS                5
REQMEM_TYPE        Mc
Name: 1000006, dtype: object


The logic we need:
* IF there are 5 CPUs per 1 Node
* AND REQMEM in mn = 15M
* THEN REQMEM per mc = 1 / 5 * 15 = 3M per c = node# / CPU# * mn#
* ELSE REQMEM_NO  

Possible code (draft, incomplete for loop) =  

    for row in accre:
    accre['REQMEM_CVT'] = 0
    
    if REQMEM_TYPE == 'Mc':
        accre['REQMEM_CVT'] = accre['REQMEM_NO']
        
    elif REQMEM_TYPE == 'Mn':
        accre['REQMEM_CVT'] = (accre['REQMEM_NO'] * (accre['NODES']/accre['CPUS']))
        
What that looks like in code =

    # If statement, tested on MC:
    if test_mc['REQMEM_TYPE'] == 'Mc':
        print(test_mc['REQMEM_NO'])
    elif test_mc['REQMEM_TYPE'] == 'Mn':
        print(test_mc['REQMEM_NO'] * (test_mc['NODES']/test_mc['CPUS']))
    
    # If statement, tested on Mn:
    if test_mn['REQMEM_TYPE'] == 'Mc':
        print(test_mn['REQMEM_NO'])
    elif test_mn['REQMEM_TYPE'] == 'Mn':
        print(test_mn['REQMEM_NO'] * (test_mn['NODES']/test_mn['CPUS']))

In [11]:
# Create a new calculated field 'REQMEM_PER_CORE' which checks the original REQMEM_TYPE before returning a value
accre['REQMEM_PER_CORE'] = np.where((accre['REQMEM_TYPE'] == 'Mn'), \
                                      (accre['REQMEM_NO'] * (accre['NODES']/accre['CPUS'])), \
                                       accre['REQMEM_NO'])
# Check to see if it worked
accre.head(200)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.000000
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.000000
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000000
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000000
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.000000
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn,5000.000000
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc,2048.000000
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000000
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000000
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn,7000.000000


In [12]:
# Create a new column USEDMEM_NO which only contains the number
accre['USEDMEM_NO'] = accre['USEDMEM'].str.extract('(\d+)')
# Create a new column USEDMEM_TYPE which only contains the type (in case there are multiple, which there are not)
accre['USEDMEM_TYPE'] = accre['USEDMEM'].str.extract('(\D$)')
# Convert USEDMEM_NO to a mathable data type
accre['USEDMEM_NO'] = pd.to_numeric(accre['USEDMEM_NO'])
# Create new calculated field USEDMEM_PER_CORE
accre['USEDMEM_PER_CORE'] = accre['USEDMEM_NO'] / accre['CPUS']

In [13]:
# Check that the dataframe looks right with all of the new columns
accre.head(500)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_NO,USEDMEM_TYPE,USEDMEM_PER_CORE
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.000000,1284,M,1284.000000
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.000000,1275,M,1275.000000
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000000,1259,M,1259.000000
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000000,1271,M,1271.000000
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.000000,1260,M,1260.000000
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn,5000.000000,1258,M,1258.000000
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc,2048.000000,828,M,828.000000
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000000,2846,M,711.500000
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000000,9838,M,2459.500000
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn,7000.000000,4915,M,1228.750000


Next, convert the fields ``REQTIME`` and ``USEDTIME`` to seconds.

In [14]:
# Create a function to split the hh:mm:ss string and calculate seconds from it
def to_sec(x):
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

In [15]:
# Separate out the days from REQTIME, put them in another column, convert to a number, fill NaN with 0, convert to seconds
accre['REQTIME_DAY_SEC'] = accre['REQTIME'].str.extract('(\d+)-')
accre['REQTIME_DAY_SEC'] = pd.to_numeric(accre['REQTIME_DAY_SEC'])
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC'].fillna(0)
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC']*24*60*60
# Extract the hh:mm:ss from REQTIME, put in a new column, and then apply the to_sec function 
accre['REQTIME_T'] = accre['REQTIME'].str.extract('(..:..:..)$')
# REQTIME_SEC includes total seconds from REQTIME
accre['REQTIME_SEC'] = accre['REQTIME_T'].apply(to_sec) + accre['REQTIME_DAY_SEC']
# Do the same for USEDTIME
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME'].str.extract('(\d+)-')
accre['USEDTIME_DAY_SEC'] = pd.to_numeric(accre['USEDTIME_DAY_SEC'])
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC'].fillna(0)
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC']*24*60*60
# Do the same for USEDTIME
accre['USEDTIME_T'] = accre['USEDTIME'].str.extract('(..:..:..)$')
# USEDTIME_SEC includes total second from USEDTIME
accre['USEDTIME_SEC'] = accre['USEDTIME_T'].apply(to_sec) + accre['USEDTIME_DAY_SEC']
# Check to make sure the data types look okay
# accre.info()

In [16]:
# Thinking about alternate approaches
# accre['REQTIME_SEC'] = np.where(accre['REQTIME'].re.findall(r'(\d+)-' == True, 'yes', 'no')

In [17]:
accre.head(500)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_NO,USEDMEM_TYPE,USEDMEM_PER_CORE,REQTIME_DAY_SEC,REQTIME_T,REQTIME_SEC,USEDTIME_DAY_SEC,USEDTIME_T,USEDTIME_SEC
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.000000,1284,M,1284.000000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.000000,1275,M,1275.000000,172800.0,00:00:00,172800.0,0.0,00:11:41,701.0
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000000,1259,M,1259.000000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000000,1271,M,1271.000000,172800.0,00:00:00,172800.0,0.0,00:15:36,936.0
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.000000,1260,M,1260.000000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn,5000.000000,1258,M,1258.000000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc,2048.000000,828,M,828.000000,0.0,12:00:00,43200.0,0.0,00:05:08,308.0
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000000,2846,M,711.500000,172800.0,00:00:00,172800.0,0.0,01:12:11,4331.0
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000000,9838,M,2459.500000,172800.0,00:00:00,172800.0,86400.0,22:25:09,167109.0
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn,7000.000000,4915,M,1228.750000,172800.0,00:00:00,172800.0,0.0,20:12:20,72740.0


In [18]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 1000000 to 1004999
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   JOBID             5000 non-null   object 
 1   ACCOUNT           5000 non-null   object 
 2   USER              5000 non-null   object 
 3   REQMEM            5000 non-null   object 
 4   USEDMEM           5000 non-null   object 
 5   REQTIME           5000 non-null   object 
 6   USEDTIME          5000 non-null   object 
 7   NODES             5000 non-null   int64  
 8   CPUS              5000 non-null   int64  
 9   PARTITION         5000 non-null   object 
 10  EXITCODE          5000 non-null   object 
 11  STATE             5000 non-null   object 
 12  NODELIST          5000 non-null   object 
 13  REQMEM_NO         5000 non-null   int64  
 14  REQMEM_TYPE       5000 non-null   object 
 15  REQMEM_PER_CORE   5000 non-null   float64
 16  USEDMEM_NO        5000 non-null  